In [3]:
from pinecone import Pinecone,ServerlessSpec
import google.generativeai as genai
import pandas as pd
import time
import os
import dotenv 
dotenv.load_dotenv()

True

In [4]:
api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)

embedding_model_name = os.getenv("EMBEDDING_MODEL_NAME", "models/text-embedding-004")
model_name = os.getenv("MODEL_NAME", "gemini-1.5-flash") 
pinecone_api_key = os.getenv("PINECONE_API_KEY") 
index_name = os.getenv("PINECONE_INDEX_NAME")

In [5]:
pc = Pinecone(api_key=pinecone_api_key)

In [6]:
def get_embedding(data):
    result = genai.embed_content(
        model=embedding_model_name , 
        content=data
    )

    embedding = result['embedding']
    return embedding

### Try embedding

In [7]:
data= """hello_world"""
data_embeddings = get_embedding(data)
print(data_embeddings)

[0.027252177, 0.0006593643, -0.042983446, 0.0033653285, -0.008581633, -0.023700666, 0.04640355, 0.010048851, 0.030217871, 0.05144418, -0.052055273, 0.021806804, 0.04318359, -0.0076872665, -0.01084935, -0.036215037, 4.242308e-05, 0.020921998, -0.09486348, 0.022864625, 0.0047533596, -0.0031489993, -0.031189984, -0.05277918, -0.0077846157, 0.02698742, -0.009399465, 0.026942307, 0.01900396, 0.035722855, -0.024693247, 0.033065196, -0.013609282, -0.043306142, -0.008054359, 0.010896698, -0.056849964, 0.028405467, -0.009594443, -0.08026029, -0.01713169, 0.043653037, 0.021905644, 0.023178073, 0.057423994, -0.0028906565, 0.023469498, -0.011805368, 0.005437563, 0.016367037, 0.045914188, 0.015137211, -0.09489855, 0.050591193, -0.03115714, 0.0029970703, -0.02756301, -0.046763547, 0.122208394, -0.01842202, -0.05428407, -0.05603495, 0.03407623, 0.0042298352, -0.029316427, -0.028413309, -0.0087376805, 0.026752718, -0.047331944, 0.025705762, -0.054655768, -0.006202983, -0.042814348, 0.015243739, -0.054

In [8]:
len(data_embeddings)

768

### Our Dataset :

In [9]:
df=pd.read_json('products/products.jsonl',lines=True)
df.head()

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp
2,Latte,Coffee,"Smooth and creamy, our latte combines rich esp...","[Espresso, Steamed Milk, Milk Foam]",4.75,4.8,Latte.jpg
3,Chocolate Chip Biscotti,Bakery,"Crunchy and delightful, this chocolate chip bi...","[Flour, Sugar, Chocolate Chips, Eggs, Almonds,...",2.50,4.6,chocolat_biscotti.jpg
4,Espresso shot,Coffee,"A bold shot of rich espresso, our espresso is ...",[Espresso],2.00,4.9,Espresso_shot.webp


In [10]:
df['text'] =  df['name']+" : "+df['description'] + \
                " -- Ingredients: " + df['ingredients'].astype(str) + \
                " -- Price: " + df['price'].astype(str) + \
                " -- rating: " + df['rating'].astype(str) 

df['text'].head()

0    Cappuccino : A rich and creamy cappuccino made...
1    Jumbo Savory Scone : Deliciously flaky and but...
2    Latte : Smooth and creamy, our latte combines ...
3    Chocolate Chip Biscotti : Crunchy and delightf...
4    Espresso shot : A bold shot of rich espresso, ...
Name: text, dtype: object

In [11]:
texts = df['text'].tolist()
texts

["Cappuccino : A rich and creamy cappuccino made with freshly brewed espresso, steamed milk, and a frothy milk cap. This delightful drink offers a perfect balance of bold coffee flavor and smooth milk, making it an ideal companion for relaxing mornings or lively conversations. -- Ingredients: ['Espresso', 'Steamed Milk', 'Milk Foam'] -- Price: 4.5 -- rating: 4.7",
 "Jumbo Savory Scone : Deliciously flaky and buttery, this jumbo savory scone is filled with herbs and cheese, creating a mouthwatering experience. Perfect for a hearty snack or a light lunch, it pairs beautifully with your favorite coffee or tea. -- Ingredients: ['Flour', 'Butter', 'Cheese', 'Herbs', 'Baking Powder', 'Salt'] -- Price: 3.25 -- rating: 4.3",
 "Latte : Smooth and creamy, our latte combines rich espresso with velvety steamed milk, creating a perfect balance of flavor and texture. Enjoy it as a comforting treat any time of day, whether you're starting your morning or taking a midday break. -- Ingredients: ['Espre

In [12]:
with open('products/Merry\'s_way_about_us.txt') as f:
    Merry_way_about_section = f.read()
    
Merry_way_about_section = "Coffee shop Merry's Way about section: " + Merry_way_about_section
Merry_way_about_section

"Coffee shop Merry's Way about section: Welcome to Merry's Way Coffee, your neighborhood coffee shop located in the heart of Greenwich Village, New York City. At Merry's Way, we believe that coffee is more than just a drinkâ€”itâ€™s an experience, a moment of joy, and a way to connect with others.\n\nOur Story\nFounded in 2015, Merryâ€™s Way started as a small family-owned cafÃ© with one mission: to share the love of quality, ethically-sourced coffee with our community.\n\nMerry's passion for travel and coffee led her on a journey across South America, where she handpicked partnerships with small farms and cooperatives. We ensure that every cup we brew tells a story of dedication and care, from farm to table. Our beans are roasted in-house to bring out unique flavors that reflect the regions where they were grown.\n\nDelivery & Locations Served\nIn addition to offering a cozy place to enjoy coffee in our cafÃ©, we proudly deliver to Greenwich Village, SoHo, West Village, and Lower Manh

In [13]:
with open('products/menu_items_text.txt') as f:
    menue_items_text = f.read()
    
menue_items_text = "Menu Items: " + menue_items_text
menue_items_text

'Menu Items: Menu Items\n\nCappuccino - $4.50\nJumbo Savory Scone - $3.25\nLatte - $4.75\nChocolate Chip Biscotti - $2.50\nEspresso shot - $2.00\nHazelnut Biscotti - $2.75\nChocolate Croissant - $3.75\nDark chocolate (Drinking Chocolate) - $5.00\nCranberry Scone - $3.50\nCroissant - $3.25\nAlmond Croissant - $4.00\nGinger Biscotti - $2.50\nOatmeal Scone - $3.25\nGinger Scone - $3.50\nChocolate syrup - $1.50\nHazelnut syrup - $1.50\nCarmel syrup - $1.50\nSugar Free Vanilla syrup - $1.50\nDark chocolate (Packaged Chocolate) - $3.00'

In [14]:
texts.append(Merry_way_about_section)
texts.append(menue_items_text)

In [15]:
texts

["Cappuccino : A rich and creamy cappuccino made with freshly brewed espresso, steamed milk, and a frothy milk cap. This delightful drink offers a perfect balance of bold coffee flavor and smooth milk, making it an ideal companion for relaxing mornings or lively conversations. -- Ingredients: ['Espresso', 'Steamed Milk', 'Milk Foam'] -- Price: 4.5 -- rating: 4.7",
 "Jumbo Savory Scone : Deliciously flaky and buttery, this jumbo savory scone is filled with herbs and cheese, creating a mouthwatering experience. Perfect for a hearty snack or a light lunch, it pairs beautifully with your favorite coffee or tea. -- Ingredients: ['Flour', 'Butter', 'Cheese', 'Herbs', 'Baking Powder', 'Salt'] -- Price: 3.25 -- rating: 4.3",
 "Latte : Smooth and creamy, our latte combines rich espresso with velvety steamed milk, creating a perfect balance of flavor and texture. Enjoy it as a comforting treat any time of day, whether you're starting your morning or taking a midday break. -- Ingredients: ['Espre

> - ### Now data is ready for embedding

### Generate Embeddings

In [16]:
embeddings = get_embedding(texts)
embeddings

[[0.005776356,
  -0.008747355,
  -0.022011248,
  -0.00062965415,
  0.026473481,
  0.021557178,
  0.008611353,
  0.0039576134,
  0.01658484,
  0.031983837,
  -0.021553667,
  0.119529516,
  0.022053061,
  0.042518083,
  0.060015198,
  -0.085740514,
  0.006636195,
  0.11056146,
  -0.0837044,
  0.0015353948,
  0.011290049,
  0.028805183,
  0.0061612036,
  -0.013588416,
  -0.03452312,
  -0.014129162,
  -0.054784328,
  0.032538086,
  0.012460008,
  -0.010107406,
  -0.015650727,
  -0.007404588,
  0.005189827,
  -0.025629498,
  -0.022394724,
  0.008241333,
  -0.048609447,
  0.0022239108,
  0.0017600502,
  -0.060456336,
  -0.065884925,
  -0.035643034,
  -0.030917205,
  0.060103808,
  0.01644082,
  0.03490382,
  -0.045494493,
  0.022165187,
  -0.004698517,
  0.048953604,
  0.05368573,
  0.008534005,
  0.024417354,
  -0.04080428,
  -0.044911508,
  -0.014576308,
  -0.01244415,
  0.045557186,
  0.05241548,
  -0.014259347,
  0.0063168393,
  -0.043098,
  0.040191025,
  -0.016696237,
  0.031225968,
  

### Push embeddings to database :

In [67]:
index_name = "ourcoffeeshop"

pc.create_index(
    name=index_name,
    dimension=768, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [19]:
# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

vectors = []
for text, e in zip(texts, embeddings):
    entry_id = text.split(":")[0].strip()

    vectors.append({
        "id": entry_id,
        "values": e, 
        "metadata": {'text': text}
    })

index.upsert(
    vectors=vectors,
    namespace="ns1"
)

{'upserted_count': 20}

 ###  Done!!

### Check and get Closest :

In [20]:
user_query = """Is Cappuccino lactose-free?"""
user_query_embeddings =  get_embedding(user_query)
user_query_embeddings

[0.000107461834,
 -0.0048963684,
 -0.046517454,
 -0.04684654,
 -0.010465292,
 -0.0076623424,
 0.017638382,
 0.0026871765,
 0.0135359075,
 0.07266981,
 -0.00021518391,
 0.037441954,
 -0.014972453,
 0.056340408,
 0.0153606655,
 -0.034944743,
 -0.00066328596,
 0.063193664,
 -0.04211698,
 0.023545299,
 0.044420373,
 0.041460343,
 0.015690856,
 0.00489647,
 -0.014102141,
 0.022471748,
 -0.027407227,
 0.0071784337,
 -0.028609456,
 -0.06737296,
 -0.038512643,
 0.004499665,
 0.044617686,
 -0.0033956727,
 -0.005967258,
 -0.028217988,
 -0.031370297,
 0.10526024,
 -0.012862275,
 -0.07947859,
 -0.03663647,
 -0.014025969,
 0.009752303,
 0.036039308,
 0.023255855,
 0.076756835,
 0.025988296,
 0.080879696,
 0.010001181,
 0.07166894,
 0.090599336,
 0.062542304,
 -0.017845996,
 -0.0454511,
 -0.03343206,
 0.02084078,
 -0.030067902,
 -0.012033427,
 0.021534692,
 0.028814375,
 0.0079341065,
 -0.06902888,
 -0.00035627498,
 0.00311543,
 0.006776081,
 -0.061521433,
 -0.025524406,
 -0.0010361954,
 -0.05635649

In [21]:
index_name = "ourcoffeeshop"

results = index.query(
    namespace="ns1",
    vector=user_query_embeddings,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)

{'matches': [{'id': 'Cappuccino',
              'metadata': {'text': 'Cappuccino : A rich and creamy cappuccino '
                                   'made with freshly brewed espresso, steamed '
                                   'milk, and a frothy milk cap. This '
                                   'delightful drink offers a perfect balance '
                                   'of bold coffee flavor and smooth milk, '
                                   'making it an ideal companion for relaxing '
                                   'mornings or lively conversations. -- '
                                   "Ingredients: ['Espresso', 'Steamed Milk', "
                                   "'Milk Foam'] -- Price: 4.5 -- rating: 4.7"},
              'score': 0.629026234,
              'values': []},
             {'id': 'Latte',
              'metadata': {'text': 'Latte : Smooth and creamy, our latte '
                                   'combines rich espresso with velvety '
                 

### Testing 

In [22]:
import google.generativeai as genai
import os
import numpy as np
from dotenv import load_dotenv 
import json
from sklearn.metrics.pairwise import cosine_similarity

load_dotenv()

True

In [23]:
api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)

model_name = os.getenv("MODEL_NAME", "gemini-1.5-flash") 

In [24]:
model = genai.GenerativeModel(model_name)

In [25]:
def get_chatbot_response(messages):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "parts": message["parts"]})
    response = model.generate_content(messages)  
    
    return response

In [26]:
def get_embedding(text_input):
    result = genai.embed_content(
        model=embedding_model_name , 
        content=text_input
    )

    embedding = result['embedding']
    return embedding


In [27]:
messages= []
system_prompt =  """
            You are a helpful AI assistant for a coffee shop application.
            Your task is to determine what agent should handle the user input. You have 3 agents to choose from:
            1. details_agent: This agent is responsible for answering questions about the coffee shop, like location, delivery places, working hours, details about menue items. Or listing items in the menu items. Or by asking what we have.
            2. order_taking_agent: This agent is responsible for taking orders from the user. It's responsible to have a conversation with the user about the order untill it's complete.
            3. recommendation_agent: This agent is responsible for giving recommendations to the user about what to buy. If the user asks for a recommendation, this agent should be used.

            Your output should be in a structured json format like so. each key is a string and each value is a string. Make sure to follow the format exactly:
            {
            "chain of thought": go over each of the agents above and write some your thoughts about what agent is this input relevant to.
            "decision": "details_agent" or "order_taking_agent" or "recommendation_agent". Pick one of those. and only write the word.
            "message": leave the message empty.
            }
            """
prompt = input("User: ")
messages.append({"role": "user", "parts": prompt})


input_messages = [{"role": "model", "parts": system_prompt}] + messages   

In [43]:
messages

[{'role': 'user', 'parts': 'what is the price of latte?'}]

In [45]:
messages[-1]

{'role': 'user', 'parts': 'what is the price of latte?'}

In [46]:
messages[-1]['parts']

'what is the price of latte?'

In [47]:
user_message = messages[-1]['parts']
user_message

'what is the price of latte?'

In [48]:
embeddings = get_embedding(user_message)
embeddings

[0.04742236,
 -0.030110596,
 -0.04871688,
 -0.013769747,
 -0.0002974441,
 0.008682403,
 0.004843083,
 -0.012600301,
 0.019059516,
 0.036189925,
 0.012042498,
 0.007824089,
 -0.023741549,
 0.01239802,
 0.0335447,
 -0.089161254,
 -0.0056302478,
 0.032700755,
 -0.08628341,
 -0.014475477,
 0.028294051,
 0.0040069916,
 -0.014884919,
 0.02939375,
 -0.01203726,
 -0.012316526,
 -0.033115316,
 0.0017699872,
 0.006303726,
 -0.06263927,
 -0.01953992,
 0.0058919466,
 0.042476334,
 0.002476716,
 0.010187524,
 0.018602911,
 -0.04128591,
 0.031022528,
 -0.004188412,
 -0.061570067,
 -0.029870383,
 0.039686155,
 -0.012908293,
 0.08711766,
 0.024898702,
 0.013457601,
 -0.006819931,
 0.0024263987,
 0.008513713,
 0.014145451,
 0.057014946,
 0.017156761,
 -0.014317399,
 -0.023011515,
 -0.02889391,
 0.025964025,
 -0.00726019,
 0.025605131,
 0.041418057,
 0.041380838,
 0.00779706,
 -0.0055771074,
 -0.0007863362,
 0.0051912065,
 0.03325953,
 -0.025528494,
 0.05758649,
 0.010411274,
 -0.046244346,
 -0.02312825

In [49]:
def get_closest_results(index_name,input_embeddings,top_k=2):
        index = pc.Index(index_name)
        
        results = index.query(
            namespace="ns1",
            vector=input_embeddings,
            top_k=top_k,
            include_values=False,
            include_metadata=True
        )

        return results

In [51]:
index_name = "ourcoffeeshop"

In [52]:
results = get_closest_results(index_name,embeddings,top_k=2)
results

{'matches': [{'id': 'Latte',
              'metadata': {'text': 'Latte : Smooth and creamy, our latte '
                                   'combines rich espresso with velvety '
                                   'steamed milk, creating a perfect balance '
                                   'of flavor and texture. Enjoy it as a '
                                   'comforting treat any time of day, whether '
                                   "you're starting your morning or taking a "
                                   "midday break. -- Ingredients: ['Espresso', "
                                   "'Steamed Milk', 'Milk Foam'] -- Price: "
                                   '4.75 -- rating: 4.8'},
              'score': 0.674715519,
              'values': []},
             {'id': 'Menu Items',
              'metadata': {'text': 'Menu Items: Menu Items\n'
                                   '\n'
                                   'Cappuccino - $4.50\n'
                                 

In [53]:
source_knowledge = "\n".join([x['metadata']['text'].strip()+'\n' for x in results['matches'] ])
source_knowledge

"Latte : Smooth and creamy, our latte combines rich espresso with velvety steamed milk, creating a perfect balance of flavor and texture. Enjoy it as a comforting treat any time of day, whether you're starting your morning or taking a midday break. -- Ingredients: ['Espresso', 'Steamed Milk', 'Milk Foam'] -- Price: 4.75 -- rating: 4.8\n\nMenu Items: Menu Items\n\nCappuccino - $4.50\nJumbo Savory Scone - $3.25\nLatte - $4.75\nChocolate Chip Biscotti - $2.50\nEspresso shot - $2.00\nHazelnut Biscotti - $2.75\nChocolate Croissant - $3.75\nDark chocolate (Drinking Chocolate) - $5.00\nCranberry Scone - $3.50\nCroissant - $3.25\nAlmond Croissant - $4.00\nGinger Biscotti - $2.50\nOatmeal Scone - $3.25\nGinger Scone - $3.50\nChocolate syrup - $1.50\nHazelnut syrup - $1.50\nCarmel syrup - $1.50\nSugar Free Vanilla syrup - $1.50\nDark chocolate (Packaged Chocolate) - $3.00\n"

In [56]:
prompt = f"""
        Using the contexts below, answer the query.

        Contexts:
        {source_knowledge}

        Query: {user_message}
        """

system_prompt = """ You are a customer support agent for a coffee shop called Merry's way. You should answer every question as if you are waiter and provide the neccessary information to the user regarding their orders """
messages[-1]['parts'] = prompt
input_messages = [{"role": "model", "parts": system_prompt}] + messages[-3:]


In [61]:
chatbot_output =get_chatbot_response(input_messages).text
chatbot_output

'Hello there!  The Latte is $4.75.  Anything else I can get for you?\n'

In [64]:
def postprocess(output):
        output = {
            "role": "assistant",
            "parts": output,
            "memory": {"agent":"details_agent"
                      }
        }
        return output

In [65]:
output = postprocess(chatbot_output)
output

{'role': 'assistant',
 'parts': 'Hello there!  The Latte is $4.75.  Anything else I can get for you?\n',
 'memory': {'agent': 'details_agent'}}